# Local Configuration

## Initiate Spark

In [1]:
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Create a new Spark Session
spark = SparkSession \
       .builder \
       .master("local[4]") \
       .appName("Movies") \
       .config("spark.jars", "spark-measure_2.11-0.17.jar") \
       .getOrCreate()

# Create spark metrics object
stagemetrics = StageMetrics(spark)

In [2]:
spark

## Load Datasets To Dataframes

### Movie Dataframe

In [3]:
movie_df = (spark
           .read
           .format("csv")
           .option("header", "true")
           .option("delimiter", ",")
           .option("inferSchema", "true")
           .load("datasets/movie.csv")
           )

In [4]:
movie_df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [5]:
movie_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



### Rating Dataframe

In [6]:
rating_df = (spark
            .read
            .format("csv")
            .option("header", "true")
            .option("delimiter", ",")
            .option("inferSchema", "true")
            .load("datasets/rating.csv")
            )

In [7]:
rating_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
rating_df.show(5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



### Tag Dataframe

In [9]:
tag_df = (spark
         .read
         .format("csv")
         .option("header", "true")
         .option("delimiter", ",")
         .option("inferSchema", "true") 
         .load("datasets/tag.csv")
         )

In [10]:
tag_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [11]:
tag_df.show(5)

+------+-------+-------------+-------------------+
|userId|movieId|          tag|          timestamp|
+------+-------+-------------+-------------------+
|    18|   4141|  Mark Waters|2009-04-24 18:19:40|
|    65|    208|    dark hero|2013-05-10 01:41:18|
|    65|    353|    dark hero|2013-05-10 01:41:19|
|    65|    521|noir thriller|2013-05-10 01:39:43|
|    65|    592|    dark hero|2013-05-10 01:41:18|
+------+-------+-------------+-------------------+
only showing top 5 rows



## Queries

### Query 1

In [12]:
# Start measuring performance
stagemetrics.begin()

# Get the id of the movie "Jumanji"
jumanji_id = movie_df \
            .filter(movie_df.title.contains("Jumanji")) \
            .select("movieId") \
            .collect()[0]["movieId"]

# Get the number of users that watched "Jumanji"
query_1_result = rating_df \
                .filter(rating_df["movieId"] == jumanji_id) \
                .select(count("movieId"))

# Show result
query_1_result.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+--------------+
|count(movieId)|
+--------------+
|         22243|
+--------------+

elapsedTime => 16567 (17 s)


In [13]:
# Show result for report
query_1_result.show()

+--------------+
|count(movieId)|
+--------------+
|         22243|
+--------------+



### Query 2

In [14]:
# Start measuring performance
stagemetrics.begin()

# Get the movieIds with tags containing the word "boring"
unique_boring_movieIds = tag_df \
                        .filter(lower(tag_df["tag"]).contains("boring")) \
                        .select("movieId") \
                        .dropDuplicates()

# Get the corresponding movie titles from movieIds in alphabetical order
query_2_results = unique_boring_movieIds \
                 .join(movie_df, "movieId", "inner") \
                 .select(movie_df.title) \
                 .sort(movie_df.title)

# Show results
query_2_results.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+--------------------+
|               title|
+--------------------+
|(500) Days of Sum...|
|101 Reykjavik (10...|
|12 Years a Slave ...|
|         1408 (2007)|
|1492: Conquest of...|
|2001: A Space Ody...|
|2010: The Year We...|
|         2046 (2004)|
|     21 Grams (2003)|
|24 Hour Party Peo...|
|3-Iron (Bin-jip) ...|
|40-Year-Old Virgi...|
|    6 Bullets (2012)|
| 633 Squadron (1964)|
| 7 Plus Seven (1970)|
|      8 Women (2002)|
|A.I. Artificial I...|
|  About a Boy (2002)|
|According to Gret...|
|   Adaptation (2002)|
+--------------------+
only showing top 20 rows

elapsedTime => 3056 (3 s)


In [15]:
# Show results for report
query_2_results.show(5, truncate = 50)

+------------------------------------+
|                               title|
+------------------------------------+
|         (500) Days of Summer (2009)|
|101 Reykjavik (101 Reykjavík) (2000)|
|             12 Years a Slave (2013)|
|                         1408 (2007)|
|   1492: Conquest of Paradise (1992)|
+------------------------------------+
only showing top 5 rows



### Query 3

In [16]:
# Start measuring performance
stagemetrics.begin()

# Get the userIds and movieIds with tags containing the word "bollywood"
bollywood_userIds_movieIds = tag_df \
                            .filter(lower(tag_df["tag"]).contains("bollywood")) \
                            .select(["userId", "movieId", "tag"])

# Get all userIds and movieIds with rating above 3
above_3_rating = rating_df \
                .filter(rating_df.rating > 3) \
                .select(["userId", "movieId", "rating"])

# Inner join based on unique combination of userId and movieId
query_3_results = bollywood_userIds_movieIds \
                 .join(above_3_rating, ["userId", "movieId"], "inner") \
                 .select("userId") \
                 .dropDuplicates() \
                 .sort(above_3_rating.userId)

# Show results
query_3_results.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+------+
|userId|
+------+
| 10573|
| 19837|
| 23333|
| 25004|
| 31338|
| 33323|
| 35170|
| 40514|
| 41165|
| 48816|
| 51539|
| 54900|
| 63618|
| 65908|
| 70279|
| 77137|
| 86883|
|106755|
|124139|
|130827|
+------+
only showing top 20 rows

elapsedTime => 25285 (25 s)


In [17]:
# Show results for report
query_3_results.show(5)

+------+
|userId|
+------+
| 10573|
| 19837|
| 23333|
| 25004|
| 31338|
+------+
only showing top 5 rows



 ### Query 4

In [18]:
# Start measuring performance
stagemetrics.begin()

# Group by year and movieId and find the mean value of rating
grouped_data = rating_df \
              .withColumn("year", year(rating_df["timestamp"])) \
              .groupBy(["year", "movieId"]) \
              .agg({"rating": "mean"})
              
# Create a window to limit ratings
window = Window \
        .partitionBy(grouped_data["year"]) \
        .orderBy(grouped_data["avg(rating)"] \
        .desc())

# Limit the ratings in each group
grouped_data_limited = grouped_data \
                      .select('*', row_number().over(window).alias('row_number')) \
                      .filter(col('row_number') <= 10) \
                      .sort(["year", "avg(rating)"], ascending = [True, False])

# Get the movie titles
query_4_results = grouped_data_limited \
                 .join(movie_df, "movieId", "inner") \
                 .select(["year", "title", "avg(rating)"]) \
                 .withColumnRenamed("avg(rating)", "average_rating")

# Show results
query_4_results.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+----+--------------------+------------------+
|year|               title|    average_rating|
+----+--------------------+------------------+
|1995|Seven (a.k.a. Se7...|               5.0|
|1995|Double Life of Ve...|               4.0|
|1995|   Get Shorty (1995)|               3.0|
|1995|Fish Called Wanda...|               3.0|
|1996|Substance of Fire...| 4.708333333333333|
|1996|I Can't Sleep (J'...| 4.538461538461538|
|1996|Wallace & Gromit:...| 4.519774011299435|
|1996|Schindler's List ...|4.5161642205474015|
|1996|Thieves (Voleurs,...|               4.5|
|1996|Somebody is Waiti...|               4.5|
|1996|Wallace & Gromit:...| 4.488428745432399|
|1996|Shawshank Redempt...|4.4781818181818185|
|1996|Marvin's Room (1996)| 4.388888888888889|
|1996|   Casablanca (1942)| 4.386454183266932|
|1997|Fallen Angels (Du...|               5.0|
|1997|  Spice World (1997)|               5.0|
|1997|Leading Man, The ...|               5.0|
|1997|Dangerous Beauty ...|               5.0|
|1997|     St

In [19]:
query_4_results.filter(query_4_results.year == 2005).show(truncate=100)

+----+------------------------------------------------------------------+--------------+
|year|                                                             title|average_rating|
+----+------------------------------------------------------------------+--------------+
|2005|              Not Love, Just Frenzy (Más que amor, frenesí) (1996)|           5.0|
|2005|Life Is Rosy (a.k.a. Life Is Beautiful) (Vie est belle, La) (1987)|           5.0|
|2005|                                          Paris Was a Woman (1995)|           5.0|
|2005|                                              Married to It (1991)|           5.0|
|2005|                                             Too Much Sleep (1997)|           5.0|
|2005|                                           Fear Strikes Out (1957)|           5.0|
|2005|            Before the Fall (NaPolA - Elite für den Führer) (2004)|           5.0|
|2005|                                Gate of Heavenly Peace, The (1995)|           5.0|
|2005|               

### Query 5

In [20]:
# Start measuring performance
stagemetrics.begin()

# Get the id and title of 2015 movies
movies_2015 = movie_df \
             .filter(movie_df.title.contains("(2015)")) \
             .select(["movieId", "title"])

# Get the tags of 2015 movies
joined = movies_2015 \
        .join(tag_df, "movieId", "inner") \
        .select(["title", "tag"]) \
        .sort("title")

# Group tags by movie title and concatenate them
query_5_results = joined \
                 .groupby("title") \
                 .agg(concat_ws(", ", collect_list(joined.tag)).alias("tags"))

# Show results
query_5_results.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+--------------------+--------------------+
|               title|                tags|
+--------------------+--------------------+
|A Grain of Truth ...|Borys Lankosz, Ab...|
|A Walk in the Woo...|          Ken Kwapis|
| Advantageous (2015)|      Jennifer Phang|
|As We Were Dreami...|     based on a book|
|Average Italian (...|Marcello Macchia,...|
|Beaver Trilogy Pa...|Brad Besser, movi...|
|     Blackhat (2015)|        bkk, hackers|
|     Brooklyn (2015)| John Crowley, 1950s|
|      Chappie (2015)|AI, artificial in...|
|Chuck Norris vs C...|Ilinca Calugarean...|
|   Cinderella (2015)|Cinderella, unori...|
|Diary of a Chambe...|19th century, France|
|Digging for Fire ...|        Joe Swanberg|
|  Don Verdean (2015)|Jared Hess, Jemai...|
|         Dope (2015)|       Rick Famuyiwa|
|Drunk Stoned Bril...|Douglas Tirola, m...|
| Experimenter (2015)|Michael Almereyda...|
|Fifty Shades of G...|abuse, abuse, boo...|
|        Focus (2015)|easy to watch, go...|
|     Get Hard (2015)|          

In [21]:
# Show results for report
query_5_results.show(5, truncate=50)

+--------------------------+--------------------------------------------------+
|                     title|                                              tags|
+--------------------------+--------------------------------------------------+
|   A Grain of Truth (2015)|Borys Lankosz, Abel Korzeniowski, Borys Lankosz...|
|A Walk in the Woods (2015)|                                        Ken Kwapis|
|       Advantageous (2015)|                                    Jennifer Phang|
|As We Were Dreaming (2015)|                                   based on a book|
|    Average Italian (2015)|                           Marcello Macchia, drugs|
+--------------------------+--------------------------------------------------+
only showing top 5 rows



### Query 6

In [22]:
# Start measuring performance
stagemetrics.begin()

# Group rating by movieId and get movie title from inner join
query_6_results = rating_df \
                 .groupBy("movieId") \
                 .agg({"rating": "count"}) \
                 .join(movie_df, "movieId", "inner") \
                 .select(["title", "count(rating)"]) \
                 .sort("count(rating)", ascending=False) \
                 .withColumnRenamed("count(rating)", "total_ratings")

# Show results
query_6_results.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+--------------------+-------------+
|               title|total_ratings|
+--------------------+-------------+
| Pulp Fiction (1994)|        67310|
| Forrest Gump (1994)|        66172|
|Shawshank Redempt...|        63366|
|Silence of the La...|        63299|
|Jurassic Park (1993)|        59715|
|Star Wars: Episod...|        54502|
|   Braveheart (1995)|        53769|
|Terminator 2: Jud...|        52244|
|  Matrix, The (1999)|        51334|
|Schindler's List ...|        50054|
|    Toy Story (1995)|        49695|
|Fugitive, The (1993)|        49581|
|    Apollo 13 (1995)|        47777|
|Independence Day ...|        47048|
|Usual Suspects, T...|        47006|
|Star Wars: Episod...|        46839|
|       Batman (1989)|        46054|
|Star Wars: Episod...|        45313|
|American Beauty (...|        44987|
|Twelve Monkeys (a...|        44980|
+--------------------+-------------+
only showing top 20 rows

elapsedTime => 20491 (20 s)


In [23]:
# Show results for report
query_6_results.show(5, truncate=50)

+--------------------------------+-------------+
|                           title|total_ratings|
+--------------------------------+-------------+
|             Pulp Fiction (1994)|        67310|
|             Forrest Gump (1994)|        66172|
|Shawshank Redemption, The (1994)|        63366|
|Silence of the Lambs, The (1991)|        63299|
|            Jurassic Park (1993)|        59715|
+--------------------------------+-------------+
only showing top 5 rows



### Query 7

In [24]:
# Start measuring performance
stagemetrics.begin()

# Group by year and userId and count the ratings for each group
grouped_data = rating_df \
              .withColumn("year", year(rating_df["timestamp"])) \
              .groupBy(["year", "userId"]) \
              .agg({"rating": "count"})

# Create a window to limit ratings
window = Window \
        .partitionBy(grouped_data["year"]) \
        .orderBy(grouped_data["count(rating)"] \
        .desc())

# Limit the ratings in each group
query_7_results = grouped_data \
                 .select('*', row_number().over(window).alias('row_number')) \
                 .filter(col('row_number') <= 10) \
                 .select(["year", "userId", "count(rating)"]) \
                 .sort(["year", "count(rating)"], ascending = [True, False]) \
                 .withColumnRenamed("count(rating)", "total_ratings")

# Show results
query_7_results.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+----+------+-------------+
|year|userId|total_ratings|
+----+------+-------------+
|1995|131160|            3|
|1995| 28507|            1|
|1996| 25878|          800|
|1996|  1931|          722|
|1996| 46663|          669|
|1996|107732|          657|
|1996| 24214|          624|
|1996| 41389|          605|
|1996| 19067|          605|
|1996|  4548|          589|
|1996| 46146|          570|
|1996| 81218|          510|
|1997|124052|         1352|
|1997|128653|         1141|
|1997|  5814|          849|
|1997| 64778|          655|
|1997| 83343|          583|
|1997|101971|          559|
|1997| 34962|          526|
|1997| 19954|          522|
+----+------+-------------+
only showing top 20 rows

elapsedTime => 107075 (1.8 min)


In [25]:
# Show results for report
query_7_results.filter(query_7_results.year == 1995).sort(["year", "userId"], ascending = [True, True]).show()

+----+------+-------------+
|year|userId|total_ratings|
+----+------+-------------+
|1995| 28507|            1|
|1995|131160|            3|
+----+------+-------------+



### Query 8

In [26]:
# Start measuring performance
stagemetrics.begin()

# Keep the first genre of each movie
first_genre = movie_df \
             .select('*', split(movie_df.genres, '[|]')[0].alias("genre")) \
             .drop("genres")

# Get total ratings by movie
total_ratings_by_movie = first_genre \
                        .join(rating_df, "movieId", "inner") \
                        .groupBy(["genre", "title"]) \
                        .agg({"*": "count"}) \
                        .withColumnRenamed("count(1)", "total_ratings")

# Create a window to limit total ratings
window = Window \
        .partitionBy(total_ratings_by_movie["genre"]) \
        .orderBy(total_ratings_by_movie["total_ratings"] \
        .desc())

# Limit the total ratings in each group
query_8_results = total_ratings_by_movie \
                 .select('*', row_number().over(window).alias('row_number')) \
                 .filter(col('row_number') <= 1) \
                 .select(["genre", "title", "total_ratings"]) \
                 .sort(["genre"]) \
                 .filter(total_ratings_by_movie.genre != "(no genres listed)")

# Show results
query_8_results.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+-----------+--------------------+-------------+
|      genre|               title|total_ratings|
+-----------+--------------------+-------------+
|     Action|Jurassic Park (1993)|        59715|
|  Adventure|    Toy Story (1995)|        49695|
|  Animation|Beauty and the Be...|        35138|
|   Children|E.T. the Extra-Te...|        32685|
|     Comedy| Pulp Fiction (1994)|        67310|
|      Crime|Shawshank Redempt...|        63366|
|Documentary|Bowling for Colum...|        12280|
|      Drama|Schindler's List ...|        50054|
|    Fantasy|       Brazil (1985)|        13957|
|  Film-Noir|Maltese Falcon, T...|        12144|
|     Horror|        Alien (1979)|        30933|
|       IMAX|Encounter in the ...|           30|
|    Musical|Sound of Music, T...|        14049|
|    Mystery|Twelve Monkeys (a...|        44980|
|    Romance|Meet Joe Black (1...|         5210|
|     Sci-Fi|Mission to Mars (...|         6365|
|   Thriller|Fugitive, The (1993)|        49581|
|        War|Run Sil

In [27]:
# Show results for report
query_8_results.drop("total_ratings").show(5, truncate=100)

+---------+---------------------------------+
|    genre|                            title|
+---------+---------------------------------+
|   Action|             Jurassic Park (1993)|
|Adventure|                 Toy Story (1995)|
|Animation|      Beauty and the Beast (1991)|
| Children|E.T. the Extra-Terrestrial (1982)|
|   Comedy|              Pulp Fiction (1994)|
+---------+---------------------------------+
only showing top 5 rows



### Query 9

In [28]:
# Start measuring performance
stagemetrics.begin()

# Group by year, month, day of month, hour and movieId and find the concurrent viewers for each movie
grouped_data = rating_df \
              .withColumn("year", year(rating_df["timestamp"])) \
              .withColumn("month", month(rating_df["timestamp"])) \
              .withColumn("dayofmonth", dayofmonth(rating_df["timestamp"])) \
              .withColumn("hour", hour(rating_df["timestamp"])) \
              .groupBy(["movieId","year", "month", "dayofmonth", "hour"]) \
              .agg({"*": "count"}) \
              .withColumnRenamed("count(1)", "concurrent_viewers") \
              .filter(col("concurrent_viewers") > 1)

# Get the sum of concurrent viewers
query_9_result = grouped_data.select(sum("concurrent_viewers"))

# Show result
query_9_result.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+-----------------------+
|sum(concurrent_viewers)|
+-----------------------+
|                4281178|
+-----------------------+

elapsedTime => 148348 (2.5 min)


In [29]:
# Show result for report
query_9_result.show()

+-----------------------+
|sum(concurrent_viewers)|
+-----------------------+
|                4281178|
+-----------------------+



### Query 10

In [30]:
# Start measuring performance
stagemetrics.begin()

# Keep the first genre of each movie
first_genre = movie_df \
             .select('*', split(movie_df.genres, '[|]')[0].alias("genre")) \
             .drop("genres")

# Keep movies that are tagged as "funny"
funny_movies = first_genre \
              .join(tag_df, "movieId", "inner") \
              .filter(lower(tag_df["tag"]).contains("funny")) \
              .select("movieId", "tag", "genre") \
              .dropDuplicates(["movieId"])

# Keep movies that are rated above 3.5
good_movies = first_genre \
             .join(rating_df, "movieId", "inner") \
             .groupBy("movieId") \
             .agg({"rating": "mean"}) \
             .withColumnRenamed("avg(rating)", "average_rating") \
             .filter(col("average_rating") > 3.5) \
             .select("movieId", "average_rating")

# Keep movies that are tagged as "funny" and are rated above 3.5 and group them by genre
query_10_results = funny_movies \
                  .join(good_movies, "movieId", "inner") \
                  .groupBy("genre") \
                  .agg({"*": "count"}) \
                  .withColumnRenamed("count(1)", "total_movies") \
                  .sort("genre") \
                  .select("genre", "total_movies")

# Show results
query_10_results.show()

# Stop measuring performance
stagemetrics.end()

# Print performance metrics
print(stagemetrics.report().split('\n')[6])

+-----------+------------+
|      genre|total_movies|
+-----------+------------+
|     Action|          68|
|  Adventure|          44|
|  Animation|          26|
|   Children|           7|
|     Comedy|         226|
|      Crime|          11|
|Documentary|           6|
|      Drama|          33|
|    Fantasy|           2|
|     Horror|           1|
|    Romance|           1|
+-----------+------------+

elapsedTime => 29086 (29 s)


In [31]:
# Show results
query_10_results.show(5)

+---------+------------+
|    genre|total_movies|
+---------+------------+
|   Action|          68|
|Adventure|          44|
|Animation|          26|
| Children|           7|
|   Comedy|         226|
+---------+------------+
only showing top 5 rows

